# **Task 1.1 - Download Files Scipt**
Recieve 10-K Files from Adobe, Intel Corporation and Microsoft Corp

In [79]:
from sec_edgar_downloader import Downloader
import os
from bs4 import BeautifulSoup

dl = Downloader("Company Name", "my.email@domain.com")
companies = ["ADBE", "INTC", "MSFT"]
subdirectories = []

In [ ]:
for company in companies:
    dl.get("10-K", company, after = "1995-01-01", before = "2024-01-01", download_details = True)
    org_path = "sec-edgar-filings/"+company+"/10-K/"

    for fileName in os.listdir(org_path):
        if fileName.startswith("000"):
            newName = fileName[11:]
            if (newName[0] == '9'):
                newName = "19" + newName 
            else:
                newName = "20" + newName

            path = org_path + fileName
            path_list = os.listdir(path)
            new_path= org_path + newName

            if (os.path.exists(new_path)):
                path_dir = os.listdir(new_path)
                if ((newName[2] == "9" or newName[2] == "0") and len(path_dir) == 2):
                    if (os.path.exists(new_path+"/"+path_list[1])):
                        os.remove(new_path+"/"+path_list[1])
                    elif (os.path.exists(new_path+"/"+path_list[0])):
                        os.remove(new_path+"/"+path_list[0])
                if (len(path_list) == 2):
                    os.remove(path+"/"+path_list[1])
                os.remove(path+"/"+path_list[0])
                os.rmdir(path)
            else:
                os.rename(path, new_path)
                if (len(path_list) == 2):
                    if (newName[2] == "9" or newName[2] == "0"):
                        os.remove(new_path+"/"+path_list[1])
                    else:
                        os.remove(new_path+"/"+path_list[0])

    temp = [d for d in os.listdir(org_path) if os.path.isdir(os.path.join(org_path, d))]
    for i in range(len(temp)):
        subdirectories.append(temp[i])

## **Make Files Readable for Interpretation**

In [81]:
companyNum = 0
year = 1995
input_dirs = []
years = []
years.append(year)
output_path = "10-K Files"

for index, subdirectory in enumerate(subdirectories):
    if (index % 29 == 0 and index != 0):
        companyNum = companyNum + 1
    
    path = "sec-edgar-filings/"+companies[companyNum]+"/10-K/"+subdirectory+"/"
    curr_dir = os.listdir(path)
    path = path+curr_dir[0]
    
    # Open and read the content of file
    with open(path, 'r', encoding='utf-8') as file:
        filing_content = file.read()

    soup = BeautifulSoup(filing_content, 'html.parser')

    # Extract the entire text content
    entire_content = soup.get_text()

    #Save the content to a text file with a unique name
    output_file_name = f"/10k_{year}.txt"  # Unique name based on index

    if (year + 1 < 2024):
        year += 1
        if (year not in years):
            years.append(year)
    else:
        year = 1995
    
    if not os.path.exists(output_path):
        os.mkdir(output_path)

    company_subdir = os.path.join(output_path, companies[companyNum])
    if not os.path.exists(company_subdir):
        os.mkdir(company_subdir)

    output_path = company_subdir
    output_file_path = output_path + output_file_name
    if (output_path not in input_dirs):
        input_dirs.append(output_path)

    with open(output_file_path, 'w', encoding='utf-8') as out_file:
        out_file.write(entire_content)

# **Task 1.2**

netRev = {}
opsIncome = {}

for company in companies:
    netRev[company] = []
    opsIncome[company] = []

print(netRev)
print(opsIncome)

Looking for the factors Net Revenue and Operations Income

Keywords to Search for is "(In Millions, Except Per Share Amounts)"

Look for the first occurence of a dollar sign after finding all keywords and factors

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")
inputs = tokenizer("Einstein's theory of relativity states", return_tensors="pt").to(device)

model = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b").to(device)
assistant_model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m").to(device)
outputs = model.generate(**inputs, assistant_model=assistant_model)
tokenizer.batch_decode(outputs, skip_special_tokens=True)
["Einstein's theory of relativity states that the speed of light is constant.    "]

# **Task 1.3**

In [ ]:
from dash import Dash, html, dcc, callback, dash_table, Output, Input, State
import plotly.express as px
import dash_bootstrap_components as dbc

df = pd.read_csv("Interactions Database.csv")

app = Dash(__name__)

colors = {
    'text': '#000000'
}    

app.layout = html.Div(children=[
    html.H1(
        children='Gestational Diabetes Drug-Drug Interactions',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div([
        dcc.Dropdown(
            companies, 
            placeholder = "Select a Company",
            id='companyDrop'
        ),

    html.Div(dcc.Graph(id='revenue')),
    html.Div(dcc.Graph(id='operationIncome'))
    ])
])

# @callback(
#     Output('revenue', 'figure'),
#     Input('companyID', 'value')
# )

# def updateMedicationGraph(currInteraction):
#     temp = df.loc[df['Observed Drug-Drug Interaction'] == currInteraction]
#     data = temp['Suspect Product Names']
#     fig = px.histogram(data, x = "Suspect Product Names", title = 'Cases of Patients that Used Drugs')
#     return fig

# @callback(
#     Output('severeInteraction', 'figure'),
#     Input('currInteraction', 'value')
# )

# def updateSeverityGraph(currInteraction):
#     temp = df.loc[df['Observed Drug-Drug Interaction'] == currInteraction]
#     seriousData = temp['Serious']
#     severeInteraction = px.histogram(seriousData, x = "Serious", title = "Severity of Drug-Drug Interactions")
#     return severeInteraction

if __name__ == '__main__':
    app.run(debug=True)